In [1]:
# paqueterías
import numpy as np
from scipy.special import erf # función de error
from math import floor, log10

# Elementos de matriz $f_{pq}$

El presente archivo provee una explicación y cálculo para los elementos de matriz de un solo cuerpo $f_{pq}$ para la molécula de hidrógeno empleando la base STO-3G. Considere el hamiltoniano electrónico en segunda cuantización para la molécula de hidrógeno es

$$ \mathcal{H} = \sum_{pq} f_{pq} a^\dagger_p a_q + \sum_{pqrs} g_{pqrs} a^\dagger_p a^\dagger_s a_r a_q $$

---

## STO-3G

In [2]:
# Szabo & Ostlund
d_Szabo = np.array([0.444635, 0.535328, 0.154329]) # coeficientes de contracción
a_Szabo = np.array([0.168856, 0.623913, 3.42525]) # exponentes orbitales Gaussianos

# Este trabajo
data = np.loadtxt("data/STO3G.csv", delimiter=",", usecols=(0,1), dtype=('str')) # cargar valores calculados
valores = dict()
for valor in data:
    valores[str(valor[0])] = float(valor[1])
    
d = np.array([valores['d1'], valores['d2'], valores['d3']]) # coeficientes de contracción
a = np.array([valores['a1_2'], valores['a2_2'], valores['a3_2']]) # exponentes orbitales Gaussianos

In [3]:
# distancia interatómica de 1.4 radios de Bohr (a0)
RA = np.array([0, 0, 0])
RB = np.array([1.4, 0, 0])

---
## Integrales

Se buscan los elementos de matriz $f_{pq}$ dados por la expresión siguiente:

$$ f_{pq} = \int \chi_p^* \left( -\frac{\nabla^2}{2} + \sum_C - \frac{Z_C}{r_C} \right) \chi_q \, \mathrm{d}\vec{\mathbf{x}} \,;
\quad
\vec{\mathbf{x}} = (\vec{\mathbf{r}}, \omega)
$$

La base de funciones a considerar es
$$ \chi_1 = \psi_+ \alpha \,, \quad
\chi_2 = \psi_+ \beta \,, \quad
\chi_3 = \psi_- \alpha \,, \quad
\chi_4 = \psi_- \beta \,,
$$

donde $\psi_\pm = c_\pm (\varphi_A \pm \varphi_B)$ con

$$ c_\pm = \frac{1}{\sqrt{2 (1 \pm S)}}\,, $$

la constante de normalización y

$$ \varphi_A = R_{1\mathrm{s}}^{k}(\vec{\mathbf{d}}, \vec{\mathbf{\alpha}}, r_A) Y_{lm}(\theta,\phi) \,,
\quad
\varphi_B = R_{1\mathrm{s}}^{k}(\vec{\mathbf{d}}, \vec{\mathbf{\beta}}, r_B) Y_{lm}(\theta,\phi)
$$

las funciones espaciales centradas en el núcleo $A$ y $B$, respectivamente. En esto último $R_{1\mathrm{s}}^{k}$ no es más que una combinación lineal de $k$ funciones Gaussianas $1\mathrm{s}$, también conocido como una $k$-contracción de Gaussianas $1\mathrm{s}$:

$$ R_{1\mathrm{s}}^{k}(\vec{\mathbf{d}}, \vec{\mathbf{\alpha}}, r) = \sum_{\nu=1}^{k} d_\nu R_{0}^{\mathrm{GTO}} (\alpha_\nu, r) \,;
\qquad \qquad
R_0^{\mathrm{GTO}} (\alpha, r) = \frac{2(2\alpha)^{3/4}}{\pi^{1/4}} \exp(-\alpha \, r^2) \,.
$$

La función $\varphi_A$ ($\varphi_B$) es entonces de la forma:

$$ \varphi_A = \sum_{\nu = 1}^{k} d_\nu \left( \frac{2\alpha_\nu}{\pi} \right)^{3/4} \exp(-\alpha_\nu \, r_A^2) \,.$$

Para simplificar notación considere

$$ G (\vec{\mathbf{r}}, \alpha_\nu, \vec{A}) = \left( \frac{2\alpha_\nu}{\pi} \right)^{3/4} g(\vec{\mathbf{r}}, \alpha_\nu, \vec{A})
\,; \qquad \qquad
g(\vec{\mathbf{r}}, \alpha_\nu, \vec{A}) = \exp(-\alpha_\nu \, r_A^2) \,.$$

En cuyo caso, la función $\varphi_A$ será reescrita como

$$ \varphi_A
= G_{\mu} (\vec{\mathbf{r}}, \vec{\mathbf{d}}, \vec{\mathbf{\alpha}}, \vec{A})
= \sum_{\nu = 1}^{k} d_{\nu \mu} G (\vec{\mathbf{r}}, \alpha_{\nu \mu}, \vec{A})
= \sum_{\nu = 1}^{k} d_{\nu \mu} \left( \frac{2\alpha_{\nu \mu}}{\pi} \right)^{3/4} \exp(-\alpha_{\nu \mu} \, r_A^2) \,.
$$

In [4]:
def GaussNorm(a:int) -> np.float64:
    """ Factor de normalización Gaussiano 1s

    Parámetro
        a : exponente Gaussiano
    """
    return np.power(2*a/np.pi, 3/4)

Los coeficientes de contracción $d_{\mu \, \nu}$ y los coeficientes Gaussianos $\alpha_\nu$ fueron calculados previamente en el cuaderno de Jupyter `(1) STO-3G.ipynb` .

<div class="alert alert-block alert-info">
    <strong>Integral de traslape, $S_{\mu \, \nu}$</strong>
</div>

Aquí $S_{\mu \, \nu} = \braket{\varphi_A|\varphi_B} = \braket{\varphi_B|\varphi_A}$ es la integral de traslape dada por

$$ \begin{align}
    S_{\mu \, \nu} 
    &= \int G_{\mu}^* (\vec{\mathbf{r}}, \vec{\mathbf{\alpha}}, \vec{A}) G_{\nu} (\vec{\mathbf{r}}, \vec{\mathbf{\beta}}, \vec{B}) \,\mathrm{d}V \\
    &= \int \sum_{p=1}^{k} d_{p \mu}^* \, G^* (\vec{\mathbf{r}}, \alpha_{p \mu}, \vec{A}) \,
        \sum_{q=1}^{k} d_{q \nu} \, G (\vec{\mathbf{r}}, \beta_{q \nu}, \vec{B}) \,\mathrm{d}V \\
    &= \sum_{p=1}^{k} \sum_{q=1}^{k} d_{p \mu}^* d_{q \nu} \int G (\vec{\mathbf{r}}, \alpha_{p \mu}, \vec{A}) \,
         G (\vec{\mathbf{r}}, \beta_{q \nu}, \vec{B}) \,\mathrm{d}V \\
    &= \sum_{p=1}^{k} \sum_{q=1}^{k} d_{p \mu}^* d_{q \nu} S_{pq}
\end{align}
$$

Esta última integral de acuerdo con Szabo y Ostlund es:

$$ \begin{align}
    S_{pq}
    &= \int G (\vec{\mathbf{r}}, \alpha_{p \mu}, \vec{A}) \, G (\vec{\mathbf{r}}, \beta_{q \nu}, \vec{B}) \,\mathrm{d}V \\
    &= \left( \frac{2\alpha_{p \mu}}{\pi} \right)^{3/4} \left( \frac{2\beta_{q \nu}}{\pi} \right)^{3/4}
        \int \exp(-\alpha_{p \mu} r^2_A) \exp(-\beta_{q \nu} \, r^2_B) \,\mathrm{d}V \\
    &= \left( \frac{2\alpha_{p \mu}}{\pi} \right)^{3/4} \left( \frac{2\beta_{q \nu}}{\pi} \right)^{3/4}
        \left( \frac{\pi}{\alpha_{p \mu} + \beta_{q \nu}} \right)^{3/2} \exp\left( -\frac{\alpha_{p \mu} \, \beta_{q \nu}}{\alpha_{p \mu} + \beta_{q \nu}} |\mathbf{R}_A - \mathbf{R}_B|^2 \right)
\end{align}
$$

Dentro del código esta última integral será dada por la función `Spq`.
<br>

El término exponencial aparece a lo largo de las integrales por calcular, motivo por el cual considere las siguientes convenciones:

$$ K = \exp(\mathrm{arg})\,, \qquad \mathrm{arg} = -\frac{a b}{a + b} |\mathbf{R}_A - \mathbf{R}_B|^2 .$$

Valores que serán representados en el código por las funciones `K` y `arg`, respectivamente.

In [5]:
def arg(a:float, b:float, RA:np.array, RB:np.array) -> np.float64:
    """ Argumento del factor pre-exponencial K

    Parámetros
        (a, b) : exponente orbital Gaussiano
        (RA, RB) : coordenada del núcleo (A, B)
    """
    p = a+b # exponente total
    mu = a*b/p
    RAB2 = np.square(np.linalg.norm(RA-RB))
    return -mu*RAB2

In [6]:
def K(a:float, b:float, RA:np.ndarray, RB:np.ndarray) -> np.float64:
    """ Factor pre-exponencial

    Parámetros
        (a, b) : exponente orbital Gaussiano
        (RA, RB) : coordenada del núcleo (A, B)
    """
    return np.exp(arg(a, b, RA, RB))

La expresión final de la integral de traslape $S_{\mu \nu}$ es

$$ S_{\mu \nu} 
=  \sum_{p=1}^{k} \sum_{q=1}^{k} d_{p \mu}^* d_{q \nu}  S_{pq}\,;
\qquad
S_{pq} = \left( \frac{2\alpha_{p \mu}}{\pi} \right)^{3/4} \left( \frac{2\beta_{q \nu}}{\pi} \right)^{3/4}
        \left( \frac{\pi}{\alpha_{p \mu} + \beta_{q \nu}} \right)^{3/2} \exp\left( -\frac{\alpha_{p \mu} \, \beta_{q \nu}}{\alpha_{p \mu} + \beta_{q \nu}} |\mathbf{R}_A - \mathbf{R}_B|^2 \right) \,,$$

con $\mathbf{R}_A$ y $\mathbf{R}_B$ la posición del núcleo $A$ y $B$ en unidades atómicas.

In [7]:
def Spq(a:float, b:float, RA:np.ndarray, RB:np.ndarray) -> np.float64:
    """ Integral de traslape S_pq (normalizada)
    
    Parámetros
        (a, b) : exponente orbital Gaussiano
        (RA, RB) : coordenada del núcleo (A, B)
    """
    return GaussNorm(a) * GaussNorm(b) * np.power(np.pi/(a+b), 3/2) * K(a, b, RA, RB)

In [8]:
def Smn(d:np.ndarray, a:np.ndarray, RA:np.ndarray, RB:np.ndarray) -> np.float64:
    """ Integral de traslape total S_mn
    
    Parámetros
        d : vector de coeficientes de expansión (d1, d2, ..., dk)
        a : vector de exponentes orbitales Gaussianos (a1, a2, ..., ak)
        (RA, RB) : coordenada del núcleo (A, B)
    """
    Mmn = 0 # elemento de matriz
    k = len(d)
    for p in range(k):
        for q in range(k):
            Mmn += d[p] * d[q] * Spq(a[p], a[q], RA, RB) # elemento de matriz
    return Mmn

In [9]:
SAA = Smn(d, a, RA, RA) # elemento de matriz S_AA y S_BB
SAB = Smn(d, a, RA, RB) # elemento de matriz S_AB y S_BA

print("S_AA y S_BB:", SAA)
print("S_AB y S_BA:", SAB)

S_AA y S_BB: 1.0000007374927182
S_AB y S_BA: 0.6593169268818938


Los elementos de la diagonal no son más que la condición de normalización. Dado que es hasta el séptimo decimal de estos elementos que aparece un valor diferente de cero, todos los valores serán truncados a 6 cifras de precisión decimal. Recuerde que en este caso se está tomando una distancia interatómica de $R_{AB} = 1.4 a_0$, por lo que la precisión no es necesariamente igual para todas las distancias interatómicas. La función `SAB_vdecimal` se encarga de indicar el valor de precisión decimal requerido para las integrales de traslape.

La matriz de traslape $\mathbf{S}$ con una distancia interatómica de 1.4 u.a. es de la forma

$$ \boxed{\mathbf{S}
= \begin{pmatrix} S_{AA} & S_{AB} \\ S_{BA} & S_{BB} \end{pmatrix}
= \begin{pmatrix} 1.0 & 0.659317 \\ 0.659317 & 1.0 \end{pmatrix}}
$$

In [10]:
def SAB_vdecimal(d:np.ndarray, a:np.ndarray, RC:np.ndarray) -> int:
    """ Precisión decimal integral de traslape
    Determina la cifra decimal a la cual se redondearán los elementos de matriz S
    La cifra será del orden de magnitud del primer dígito diferente de cero para la condición de normalización

    Parámetros
        d : vector de coeficientes de expansión (d1, d2, ..., dk)
        a : vector de exponentes orbitales Gaussianos (a1, a2, ..., ak)
        RC : coordenada del núcleo C
    """
    SCC = Smn(d, a, RC, RC) # elemento de matriz S_AA y S_BB
    valor = abs(floor(log10(abs(float(SCC) % 1)))+1)
    return valor

<div class="alert alert-block alert-info">
        <strong>Integral cinética, $T_{\mu \, \nu}$</strong>
</div>

La integral cinética está dada por
$$T_{\mu \, \nu}  = \braket{\varphi_A| -\frac{\nabla^2}{2} |\varphi_B} = \braket{\varphi_B| -\frac{\nabla^2}{2} |\varphi_A} \,.$$

Desarrollando,
$$ T_{\mu \, \nu} 
    = \int G_{\mu}^* (\vec{\mathbf{r}}, \vec{\mathbf{\alpha}}, \vec{A}) \left( -\frac{\nabla^2}{2} \right)
        G_{\nu} (\vec{\mathbf{r}}, \vec{\mathbf{\beta}}, \vec{B}) \,\mathrm{d}V
    = \sum_{p=1}^{k} \sum_{q=1}^{k} d_{p \mu}^* d_{q \nu} \underbrace{\int G (\vec{\mathbf{r}}, \alpha_{p \mu}, \vec{A}) \,
        \left( -\frac{\nabla^2}{2} \right)
         G (\vec{\mathbf{r}}, \beta_{q \nu}, \vec{B}) \,\mathrm{d}V}_{\displaystyle T_{pq}} \,.
$$

Desarrollando para $T_{pq}$ se obtiene
$$ T_{pq}
    = \left( \frac{2\alpha_{p \mu}}{\pi} \right)^{3/4} \left( \frac{2\beta_{q \nu}}{\pi} \right)^{3/4}
        \underbrace{\int \exp(-\alpha_{p \mu} r^2_A) \left( -\frac{\nabla^2}{2} \right) \exp(-\beta_{q \nu} \, r^2_B) \,\mathrm{d}V}_{\displaystyle I_1} \,.
$$

La integral $I_1$ de la expresión previa de acuerdo con Szabo y Ostlund es:
$$ I_1 = \frac{\alpha_{p \mu} \, \beta_{q \nu}}{\alpha_{p \mu} + \beta_{q \nu}}
    \left[ 3 - \frac{2 \alpha_{p \mu} \, \beta_{q \nu}}{\alpha_{p \mu} + \beta_{q \nu}} |\mathbf{R}_A - \mathbf{R}_B|^2 \right]
    \left( \frac{\pi}{\alpha_{p \mu} + \beta_{q \nu}} \right)^{3/2}
    \exp\left( -\frac{\alpha_{p \mu} \, \beta_{q \nu}}{\alpha_{p \mu} + \beta_{q \nu}} |\mathbf{R}_A - \mathbf{R}_B|^2 \right) \,.$$

Observe que los últimos dos factores de $I_1$ multiplicados por los respectivos coeficientes de normalización no es más que $S_{pq}$, i.e.
$$ T_{pq}
    = \frac{\alpha_{p \mu} \, \beta_{q \nu}}{\alpha_{p \mu} + \beta_{q \nu}}
    \left[ 3 - \frac{2 \alpha_{p \mu} \, \beta_{q \nu}}{\alpha_{p \mu} + \beta_{q \nu}} |\mathbf{R}_A - \mathbf{R}_B|^2 \right]
    S_{pq} \,.
$$

In [11]:
def Tpq(a:np.ndarray, b:np.ndarray, RA:np.ndarray, RB:np.ndarray) -> np.float64:
    """ Integral cinética T_pq (normalizada)

    Parámetros
        d : vector de coeficientes de expansión (d1, d2, ..., dk)
        a : vector de exponentes orbitales Gaussianos (a1, a2, ..., ak)
        (RA, RB) : coordenada del núcleo (A, B)
    """
    RAB2 = np.square(np.linalg.norm(RA-RB)) # cuadrado de diferencia internuclear
    return (a*b)/(a+b) * (3 - 2*(a*b)/(a+b)*RAB2 ) * Spq(a, b, RA, RB)

De lo cual, la expresión final de la integral cinética $T_{\mu \nu}$ no es más que

$$ T_{\mu \nu} 
=  \sum_{p=1}^{k} \sum_{q=1}^{k} d_{p \mu}^* d_{q \nu}
        \frac{\alpha_{p \mu} \, \beta_{q \nu}}{\alpha_{p \mu} + \beta_{q \nu}} \left[ 3 - \frac{2 \alpha_{p \mu} \, \beta_{q \nu}}{\alpha_{p \mu} + \beta_{q \nu}} |\mathbf{R}_A - \mathbf{R}_B|^2 \right] S_{pq} \,,
$$

con $\mathbf{R}_A$ y $\mathbf{R}_B$ la posición del núcleo $A$ y $B$ en unidades atómicas.

In [12]:
def Tmn(d:np.ndarray, a:np.ndarray, RA:np.ndarray, RB:np.ndarray) -> np.float64:
    """ Integral cinética total T_mn

    Parámetros
        d : vector de coeficientes de expansión (d1, d2, ..., dk)
        a : vector de exponentes orbitales Gaussianos (a1, a2, ..., ak)
        (RA, RB) : coordenada del núcleo (A, B)
    """
    Mmn = 0 # elemento de matriz
    k = len(d) # tamaño de la k-contracción
    for p in range(k):
        for q in range(k):
            Mmn += d[p] * d[q] * Tpq(a[p], a[q], RA, RB) # elemento de matriz
    return Mmn

A modo de control se calculan los elementos de la matriz $\mathbf{T}_{AB}$ para $R_{AB} = 1.4a_0$.

In [13]:
TAA = Tmn(d, a, RA, RA) # elemento de matriz T_AA y T_BB
TAB = Tmn(d, a, RA, RB) # elemento de matriz T_AB y T_BA

print("T_AA & T_BB:", TAA)
print("T_AB & T_BA:", TAB)

T_AA & T_BB: 0.7600584704774266
T_AB & T_BA: 0.23645821621430846


La matriz cinética $\mathbf{T}_{AB}$ es de la forma

$$ \boxed{\mathbf{T}_{AB}
= \begin{pmatrix} T_{AA} & T_{AB} \\ T_{BA} & T_{BB} \end{pmatrix}
= \begin{pmatrix} 0.7600584704774266 & 0.23645821621430846 \\ 0.23645821621430846 & 0.7600584704774266 \end{pmatrix}}
$$

<div class="alert alert-block alert-info">
        <strong>Integral Coulombiana del i-ésimo núcleo, $V^i_{\mu \, \nu}$</strong>
</div>

La integral Coulombiana del $i$-ésimo núcleo está dada por

$$V^i_{\mu \, \nu}  = \braket{\varphi_A| \sum_C  - \frac{Z_C}{r_c} |\varphi_B} = \braket{\varphi_B| \sum_C  -\frac{Z_C}{r_c} |\varphi_A} \,.$$

Desarrollando,

$$ \begin{align}
    V^i_{\mu \, \nu} 
    = \int G_{\mu}^* (\vec{\mathbf{r}}, \vec{\mathbf{\alpha}}, \vec{A}) \left( \sum_C -\frac{Z_C}{r_c} \right)
        G_{\nu} (\vec{\mathbf{r}}, \vec{\mathbf{\beta}}, \vec{B}) \,\mathrm{d}V
    = \sum_{p=1}^{k} \sum_{q=1}^{k} d_{p \mu}^* d_{q \nu} \sum_C
    \underbrace{\int G^* (\vec{\mathbf{r}}, \alpha_{p \mu}, \vec{A}) \,
        \left( -\frac{Z_C}{r_c} \right)
         G (\vec{\mathbf{r}}, \beta_{q \nu}, \vec{B}) \,\mathrm{d}V}_{\displaystyle V_{pq} (C)}
\end{align}
$$

Esta última integral de acuerdo con Szabo y Ostlund es

$$ \begin{align}
    V_{pq} (C)
    &= \left( \frac{2\alpha_{p \mu}}{\pi} \right)^{3/4} \left( \frac{2\beta_{q \nu}}{\pi} \right)^{3/4}
        \left( -\frac{2\pi}{\alpha_{p \mu} + \beta_{q \nu}} \right) Z_C \,
        \exp\left( -\frac{\alpha_{p \mu} \, \beta_{q \nu}}{\alpha_{p \mu} + \beta_{q \nu}} |\mathbf{R}_A - \mathbf{R}_B|^2 \right)
        F_0 \left[ (\alpha_{p \mu} + \beta_{q \nu}) |\mathbf{R}_P - \mathbf{R}_C|^2 \right]
\end{align}
$$

donde $F_0(t)$ es la función de Boys con $n=0$ dada por

$$ F_0(t) = \frac{1}{2} \left( \frac{\pi}{t} \right)^{1/2} \mathrm{erf}(t^{1/2})
\quad
\text{con} \quad \mathrm{erf}(z) = \frac{2}{\sqrt{\pi}} \int_0^{z} e^{-t^2} \, \mathrm{d}t
$$

y $\mathbf{R}_P$ es la coordenada de centro de carga

$$ \mathbf{R}_P = \frac{\alpha \mathbf{R}_A + \beta \, \mathbf{R}_B}{\alpha + \beta} \,.$$

In [14]:
def F0(t:float) -> np.float64:
    """ Función de Boys, n=0

    Parámetro
        t : argumento de la función
    """
    return (1/2) * np.sqrt(np.pi/t) * erf(np.sqrt(t))

In [15]:
def RP(a:float, b:float, RA:np.ndarray, RB:np.ndarray) -> np.ndarray:
    """ Coordenada de centro de carga
    
    Parámetro
        (a, b) : exponente orbital Gaussiano
        (RA, RB) : coordenada del núcleo (A, B)
    """
    p = a + b # exponente total
    return (a*RA+b*RB)/p

<div class="alert alert-block alert-warning">
        <strong>Caso particular.</strong> Todos los núcleos son iguales.
</div>

$$\begin{align}
    \int \exp(-\alpha \, r^2_A) \left( -\frac{Z_A}{r_A} \right) \exp(-\beta \, r^2_A) \,\mathrm{d}V
    &= -Z_A \int \frac{\exp[ - (\alpha + \beta) \, r^2]}{r} \,\mathrm{d}V \\
    &= -Z_A \int \mathrm{d}\Omega \int r \exp[ - (\alpha + \beta) \, r^2] \,\mathrm{d}r \\
    &= - Z_A (4 \pi) \frac{1}{2(\alpha + \beta)} \\
    &= - \frac{2 \pi Z_A}{\alpha + \beta}
\end{align}
$$

In [16]:
def Vpq_AB(a:float, b:float, RA:np.ndarray, RB:np.ndarray, RC:np.ndarray, ZC:float) -> np.float64:
    """ Integral cinética V_pq(C)

    Parámetros
        (a, b) : exponente orbital Gaussiano
        (RA, RB, RC) : coordenada del núcleo (A, B, C)
        ZC : carga del núcleo C
    """
    factor = -2*np.pi/(a+b) * ZC # factor común del término pq
    
    if np.array_equal(RA, RB) and np.array_equal(RB, RC): # todos los núcleos iguales
        Vpq = factor
    else: # cualquier otro caso
        RAB2 = np.square(np.linalg.norm(RA-RB))
        Rp = RP(a, b, RA, RB) # coordenada de centro de carga
        RPC2 = np.square(np.linalg.norm(Rp-RC))
        Vpq = factor * K(a, b, RA, RB) * F0((a+b)*RPC2)
    return Vpq

La expresión final para el elemento de matriz $V^i_{\mu \, \nu}$ es

$$ V^i_{\mu \, \nu}
= \sum_{p=1}^{k} \sum_{q=1}^{k} d_{p \mu}^* d_{q \nu} \left[ V^i_{pq}(A) + V^i_{pq}(B) \right]
= \sum_{p=1}^{k} \sum_{q=1}^{k} d_{p \mu}^* d_{q \nu} V^i_{pq}(A)
    + \sum_{p=1}^{k} \sum_{q=1}^{k} d_{p \mu}^* d_{q \nu}  V^i_{pq}(B)
$$

In [17]:
def Vmn(d:np.ndarray, a:np.ndarray, RA:np.ndarray, RB:np.ndarray, RC:np.ndarray, ZC:float) -> np.float64:
    """ Integral cinética V_mn

    Parámetros
        d : vector de coeficientes de expansión (d1, d2, ..., dk)
        a : vector de exponentes orbitales Gaussianos (a1, a2, ..., ak)
        (RA, RB, RC) : coordenada del núcleo (A, B, C)
        ZC : carga del núcleo C
    """
    Mmn1 = 0 # elemento de matriz V^i_AA
    Mmn2 = 0 # elemento de matriz V^i_AB y V^i_BA
    Mmn3 = 0 # elemento de matriz V^i_BB
    k = len(d)

    for p in range(k):
        for q in range(k):
            factor = d[p] * d[q] * GaussNorm(a[p]) * GaussNorm(a[q]) # factor común del término p, q
            Mmn1 += factor * Vpq_AB(a[p], a[q], RA, RA, RC, ZC)
            Mmn2 += factor * Vpq_AB(a[p], a[q], RA, RB, RC, ZC)
            Mmn3 += factor * Vpq_AB(a[p], a[q], RB, RB, RC, ZC)
    
    return [Mmn1, Mmn2, Mmn3] # elementos de matriz (V^i_AA, V^i_AB, V^i_BB)

In [18]:
ZA, ZB = 1, 1 # cargas de los núcleos A y B

VA = Vmn(d, a, RA, RB, RA, ZA) # matriz V^A (centrada en núcleo A)
VB = Vmn(d, a, RA, RB, RB, ZB) # matriz V^B (centrada en núcleo B)

print("VA:", VA)
print("VB:", VB)

VA: [np.float64(-1.226648498439419), np.float64(-0.597421431195225), np.float64(-0.6538293950694781)]
VB: [np.float64(-0.6538293950694781), np.float64(-0.5974214311952252), np.float64(-1.226648498439419)]


Note que los valores en las antidiagonales de $\mathbf{V}^A$ y $\mathbf{V}^B$ no coinciden, no obstante estos deberían ser iguales. Por ello mismo, de forma similar que se hizo para la integral de traslape $S_{pq}$, se considerará hasta el último decimal donde estos son iguales. La función `Vmn_decimal` se encargará de determinar dicho decimal.

La matriz Coulombiana $\mathbf{V}^A$ y $\mathbf{V}^B$ son entonces

$$ \boxed{\mathbf{V}^A
= \begin{pmatrix} V^A_{AA} & V^A_{AB} \\ V^A_{BA} & V^A_{BB} \end{pmatrix}
= \begin{pmatrix} -1.226648498439419 & -0.597421431195225 \\ -0.597421431195225 & -0.6538293950694781 \end{pmatrix}}
$$

$$ \boxed{\mathbf{V}^B
= \begin{pmatrix} V^B_{AA} & V^B_{AB} \\ V^B_{BA} & V^B_{BB} \end{pmatrix}
= \begin{pmatrix} -0.6538293950694781 & -0.597421431195225 \\ -0.597421431195225 & -1.226648498439419 \end{pmatrix}}
$$

In [19]:
def ViAB_vdecimal(d:np.ndarray, a:np.ndarray, RA:np.ndarray, RB:np.ndarray) -> int:
    """ Precisión decimal integral Coulombiana
    Determina la cifra decimal a la cual se redondearán los elementos de la antidiagonal de las matrices V^A y V^B
    La cifra será del orden de magnitud del primer dígito diferente de cero para la condición de normalización

    Parámetros
        d : vector de coeficientes de expansión (d1, d2, ..., dk)
        a : vector de exponentes orbitales Gaussianos (a1, a2, ..., ak)
        (RA, RB) : coordenada del núcleo (A, B)
    """
    VA_AB = Vmn(d, a, RA, RB, RA, ZA)[1] # elemento de matriz V^A_AB
    VB_AB = Vmn(d, a, RA, RB, RB, ZB)[1] # elemento de matriz V^B_AB

    str_a, str_b = str(VA_AB), str(VB_AB) # convertir a texto
    str_a, str_b = str_a.split('.',1)[1], str_b.split('.',1)[1] # separar en parte entera y decimal (se conserva la decimal)
    len_a, len_b = len(str_a), len(str_b) # largo de cada valor
    min_len = min(len_a, len_b) # largo más pequeño para evitar IndexError

    for i in range(min_len):
        if str_a[i] != str_b[i]:
            return i # valor decimal donde difieren

    # dígitos coinciden hasta el decimal 'min_len', revisa si largos son diferentes
    if len_a != len_b:
        return min_len

    return -1 # valores son idénticos

<div class="alert alert-block alert-info">
<strong>Coeficiente de normalización, $c_{\pm}$</strong>
</div>

El coeficiente de normalización para los orbitales moleculares es

$$ c_\pm = \frac{1}{\sqrt{2(1+S_{AB})}} $$

donde $S_{AB}$ es el traslape entre el orbital atómico $\varphi_A$ y $\varphi_B$.

In [20]:
def cPM(d:np.ndarray, a:np.ndarray, RA:np.ndarray, RB:np.ndarray, signo:int) -> np.float64:
    """ Constante de normalización c±
            Psi = c± (Phi_A ± Phi_B)

    Parámetros
        d : vector de coeficientes de expansión (d1, d2, ..., dk)
        a : vector de exponentes orbitales Gaussianos (a1, a2, ..., ak)
        (RA, RB) : coordenada del núcleo (A, B)
        signo : signo de S en la normalización (1, postivo) y (-1, negativo)
    """
    SAB_decimal = SAB_vdecimal(d, a, RA) # cifra decimal de error para S_AB
    SAB_ord_mag = np.power(10, SAB_decimal) # orden de magnitud
    
    S = np.trunc(Smn(d, a, RA, RB) * SAB_ord_mag) / SAB_ord_mag # truncar S hasta el primer decimal diferente de cero para S_AA
    return 1/np.sqrt(2*(1 + signo*S))

<div class="alert alert-block alert-info">
<strong>Elemento de matriz $f_{pp}$</strong>
</div>

$$ f_{pp}
    = |c_\pm|^2 \left[ 2 ( T_{AA} \pm_p T_{AB} )
    + (V^1_{AA} + V^1_{BB}) + (V^2_{AA} + V^2_{BB})
    \pm_p 2 ( V^1_{AB} + V^2_{AB} ) \right]
$$

In [21]:
def fpp(p:int, d:np.ndarray, a:np.ndarray, RA:np.ndarray, RB:np.ndarray, ZA:float, ZB:float) -> np.float64:
    """ Elemento de matriz f_pp
    
    Parámetros
        p : elemento de la base, base = {X_1, X_2, X_3, X_4}
            p = 1, 2, 3, 4
        d : vector de coeficientes de expansión (d1, d2, ..., dk)
        a : vector de exponentes orbitales Gaussianos (a1, a2, ..., ak)
        (RA, RB) : coordenada del núcleo (A, B)
        (ZA, ZB) : carga del núcleo (A, B)
    """
    TAA = Tmn(d, a, RA, RA) # elemento de matriz T_AA y T_BB
    TAB = Tmn(d, a, RA, RB) # elemento de matriz T_AB y T_BA

    V1_AA, V1_AB, V1_BB = Vmn(d, a, RA, RB, RA, ZA) # elementos de matriz V^1_AA, V^1_AB y V^1_BB
    V2_AA, V2_AB, V2_BB = Vmn(d, a, RA, RB, RB, ZB) # elementos de matriz V^2_AA, V^2_AB y V^2_BB

    # corrección decimal términos Coulombianos
    V_decimal = ViAB_vdecimal(d, a, RA, RB) # cifra decimal de error para V^i_AB
    V1_AB = V1_AB if (V_decimal == -1) else np.round(V1_AB, V_decimal) # si -1 no hay corrección
    V2_AB = V2_AB if (V_decimal == -1) else np.round(V2_AB, V_decimal) # si -1 no hay corrección

    sgn = 1 if p in [1,2] else -1 # signo del coef. de normalización según el elemento de la base
    c2 = np.square(cPM(d, a, RA, RB, sgn)) # coef. de normalización al cuadrado
    term = c2*( 2*(TAA + sgn*TAB) + (V1_AA + V1_BB) + (V2_AA + V2_BB) + 2*sgn*(V1_AB + V2_AB) )
    return term

In [22]:
# Szabo y Oslund
f11_SO = fpp(1, d_Szabo, a_Szabo, RA, RB, ZA, ZB)
f33_SO = fpp(3, d_Szabo, a_Szabo, RA, RB, ZA, ZB)
print("--- Szabo & Ostlund ---")
print("f11 & f22:", f11_SO)
print("f33 & f44:", f33_SO)

# Este trabajo
f11 = fpp(1, d, a, RA, RB, ZA, ZB)
f33 = fpp(3, d, a, RA, RB, ZA, ZB)
print("\n--- Este trabajo ---")
print("f11 & f22:", f11)
print("f33 & f44:", f33)

--- Szabo & Ostlund ---
f11 & f22: -1.2528050205173507
f33 & f44: -0.4755920274114115

--- Este trabajo ---
f11 & f22: -1.252807825156638
f33 & f44: -0.47561604552996023
